## The Battle of Neighbourhoods (Bogota Edition!)
#### Final Assignment for applying to the IBM Data Science professional certificate
#### Juan Sebastian Mejia Perez

### Segmenting and Clustering Neighborhoods in Bogota

### Importing Libraries and necessary dependencies

In [1]:
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

import requests # library to handle requests

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
    
print('Libraries imported.')

Libraries imported.


In [2]:
import pandas as pd # library for data analsysis
print(pd.__version__)
!pip install pandas==1.0.3
print(pd.__version__)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

1.0.3
1.0.3


In [3]:
#!conda install -c conda-forge geopy --yes
print('Geopy installed')
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Geopy installed


In [4]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
print('Folium installed')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                       

### Uploading Data to Notebook

In [5]:
# Verifying actual folder
import os
print(os.getcwd())

/home/dsxuser/work


In [6]:
# Searhing for the datasets or assets preloaded on folder
os.listdir('.')

[]

In [7]:
# Confirming absolute path for reading the dataset
out = os.path.isfile("/home/dsxuser/work/Geospatial_Coordinates.csv") 
print(out)

False


**As the asset is stored as a Cloud Storage Object on the IBM Cloud it's necessary to perform the following operations to convert it, so it can be readable for the platform API**

In [8]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, 
# and used by platform APIs.
from project_lib import Project
project = Project(project_id='6e503dfd-8a63-4f1d-b358-37ad409465d6', project_access_token='p-5de9b75d6683bc94ee90d15a0f5193bc445f992a')
pc = project.project_context


In [9]:
# START CODE BLOCK
# cos2file - takes an object from Cloud Object Storage and writes it to file on container file system.
# Uses the IBM project_lib library.
# See https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/project-lib-python.html
# Arguments:
# p: project object defined in project token
# data_path: the directory to write the file
# filename: name of the file in COS

import os
def cos2file(p,data_path,filename):
    data_dir = p.project_context.home + data_path
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    open( data_dir + '/' + filename, 'wb').write(p.get_file(filename).read())

# file2cos - takes file on container file system and writes it to an object in Cloud Object Storage.
# Uses the IBM project_lib library.
# See https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/project-lib-python.html
# Arguments:
# p: prooject object defined in project token
# data_path: the directory to read the file from
# filename: name of the file on container file system

import os
def file2cos(p,data_path,filename):
    data_dir = p.project_context.home + data_path
    path_to_file = data_dir + '/' + filename
    if os.path.exists(path_to_file):
        file_object = open(path_to_file, 'rb')
        p.save_data(filename, file_object, set_project_asset=True, overwrite=True)
    else:
        print("file2cos error: File not found")
# END CODE BLOCK

In [10]:
cos2file(project, '/work', 'Geospatial_Coordinates.csv')

In [11]:
os.listdir('.')

['Geospatial_Coordinates.csv']

In [12]:
import pandas as pd
bogota_coordinates = pd.read_csv(r'Geospatial_Coordinates.csv')
bogota_coordinates

,Borough,Neighbourhood,Latitude,Longitude
0,CHAPINERO,CHICO,4.681269,-74.046840
1,CHAPINERO,CHAPINERO ALTO,4.645833,-74.063333
2,CHAPINERO,ANTIGUO COUNTRY,4.667285,-74.054726
3,CHAPINERO,MARLY,4.637717,-74.064042
4,CHAPINERO,EMAUS,4.651532,-74.055264
5,ENGATIVA,NORMANDIA,4.669283,-74.104953
6,FONTIBON,FONTIBON,4.678611,-74.141111
7,FONTIBON,MODELIA,4.667180,-74.118589
8,KENNEDY,LAS AMERICAS,4.616046,-74.136947
9,SANTA FE,SAN DIEGO,4.614470,-74.070100


### Bogota Segmenting and Clustering

### Step 1 - Getting Bogota's latitude and longitude using geopy library

In [13]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Bogota, CO'
geolocator = Nominatim(user_agent="bogota_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Bogota City are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Bogota City are 4.59808, -74.0760439.


### Step 2 - Creating Bogota City Map and adding markers to it

In [14]:
# Creates the map of Toronto using latitude and longitude values
bogota_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# Adds markers to map
for lat, lng, borough, neighbourhood in zip(bogota_coordinates['Latitude'], bogota_coordinates['Longitude'], 
                                           bogota_coordinates['Borough'], bogota_coordinates['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(bogota_map)  
    
bogota_map

### Step 3 - Provide FourSquare Credentials

In [15]:
CLIENT_ID = 'D22DNCUGEKK0HP2QPPRCM4EIPCUHFBYJEFLKI4HBAXWVJ0ND'
CLIENT_SECRET = 'NYUHPD0UYZOW0CLRV4JW51ULV3GE0CQ5UDUYVVAHQORRKFTY' 
VERSION = '20180604'

### Step 4 - Explore the first neighbourhood

In [16]:
first_boro = bogota_coordinates.loc[0, 'Borough']
first_neigh = bogota_coordinates.loc[0, 'Neighbourhood']
print(f"The first neighbourhood's name is '{first_neigh}', located in '{first_boro}'.")

The first neighbourhood's name is 'CHICO', located in 'CHAPINERO'.


In [17]:
# Obtaining neighbourhood's coordinates

first_neigh_lat = bogota_coordinates.loc[0, 'Latitude'] # neighborhood latitude value
first_neigh_long = bogota_coordinates.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(first_neigh, 
                                                               first_neigh_lat, 
                                                               first_neigh_long))

Latitude and longitude values of CHICO are 4.681269, -74.04684.


In [18]:
# Getting the top 100 venues that are within a radius of 1000 meters

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    first_neigh_lat, 
    first_neigh_long, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

In [19]:
# Defining the function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
# Clean the json and structure it into a pandas dataframe.

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Gaira,Restaurant,4.680656,-74.047639
1,Café 18,Breakfast Spot,4.680554,-74.046028
2,Good Club de Bienestar,Gym / Fitness Center,4.682004,-74.044794
3,El Dorado Bogotá,Hotel,4.680402,-74.048716
4,Chairama Spa,Spa,4.679178,-74.046655
5,Piccolo Caffe,Italian Restaurant,4.682043,-74.049302
6,Brot Bakery,Bakery,4.680833,-74.045983
7,Cosechas,Juice Bar,4.679434,-74.047076
8,CheckIn,Salad Place,4.679549,-74.047204
9,kiri cafe,Breakfast Spot,4.680567,-74.049230


### Step 5 - Exploring all neighbourhoods

First we have to create the function that will repeat the same process for all neighbourhoods

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
bogota_neighbourhood_venues = getNearbyVenues(names=bogota_coordinates['Neighbourhood'],
                                   latitudes=bogota_coordinates['Latitude'],
                                   longitudes=bogota_coordinates['Longitude']
                                  )

CHICO
CHAPINERO ALTO
ANTIGUO COUNTRY
MARLY
EMAUS
NORMANDIA
FONTIBON
MODELIA
LAS AMERICAS
SAN DIEGO
LA MACARENA
COLINA CAMPESTRE
TEUSAQUILLO
PALERMO
GALERIAS
USAQUEN
PEPE SIERRA
CEDRITOS
RINCON DEL CHICO


In [23]:
bogota_neighbourhood_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,CHICO,4.681269,-74.046840,Gaira,4.680656,-74.047639,Restaurant
1,CHICO,4.681269,-74.046840,Café 18,4.680554,-74.046028,Breakfast Spot
2,CHICO,4.681269,-74.046840,El Dorado Bogotá,4.680402,-74.048716,Hotel
3,CHICO,4.681269,-74.046840,Good Club de Bienestar,4.682004,-74.044794,Gym / Fitness Center
4,CHICO,4.681269,-74.046840,Chairama Spa,4.679178,-74.046655,Spa
5,CHICO,4.681269,-74.046840,Brot Bakery,4.680833,-74.045983,Bakery
6,CHICO,4.681269,-74.046840,CheckIn,4.679549,-74.047204,Salad Place
7,CHICO,4.681269,-74.046840,Cosechas,4.679434,-74.047076,Juice Bar
8,CHICO,4.681269,-74.046840,Piccolo Caffe,4.682043,-74.049302,Italian Restaurant
9,CHICO,4.681269,-74.046840,Escarabajos Food Bike Friends,4.681133,-74.047949,Pizza Place


Now we check how many venues were returned for each neighbourhood as follows

In [24]:
bogota_neighbourhood_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ANTIGUO COUNTRY,100,100,100,100,100,100
CEDRITOS,22,22,22,22,22,22
CHAPINERO ALTO,85,85,85,85,85,85
CHICO,28,28,28,28,28,28
COLINA CAMPESTRE,8,8,8,8,8,8
EMAUS,41,41,41,41,41,41
FONTIBON,13,13,13,13,13,13
GALERIAS,89,89,89,89,89,89
LA MACARENA,54,54,54,54,54,54


And look for how many unique categories can be curated from all the returned venues

In [25]:
print('There are {} uniques categories.'.format(len(bogota_neighbourhood_venues['Venue Category'].unique())))

There are 152 uniques categories.


### Step 6 - Analyzing each Neighbourhood

In [26]:
# one hot encoding
bogota_onehot = pd.get_dummies(bogota_neighbourhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bogota_onehot['Neighborhood'] = bogota_neighbourhood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bogota_onehot.columns[-1]] + list(bogota_onehot.columns[:-1])
bogota_onehot = bogota_onehot[fixed_columns]

bogota_onehot.head()

,Neighborhood,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gastropub,Gay Bar,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hawaiian Restaurant,Health & Beauty Service,History Museum,Hobby Shop,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Latin American Restaurant,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Multiplex,Museum,Music Store,New American Restaurant,Nightclub,Notary,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Planetarium,Plaza,Pool,Pub,Racetrack,Record Shop,Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,CHICO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,CHICO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,CHICO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,CHICO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,CHICO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
bogota_onehot.shape

(943, 153)

Group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

In [28]:
bogota_grouped = bogota_onehot.groupby('Neighborhood').mean().reset_index()
bogota_grouped

,Neighborhood,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gastropub,Gay Bar,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hawaiian Restaurant,Health & Beauty Service,History Museum,Hobby Shop,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Latin American Restaurant,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Multiplex,Museum,Music Store,New American Restaurant,Nightclub,Notary,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Planetarium,Plaza,Pool,Pub,Racetrack,Record Shop,Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,ANTIGUO COUNTRY,0.010000,0.010000,0.000000,0.000000,0.000000,0.020000,0.000000,0.010000,0.00,0.020000,0.040000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00000,0.010000,0.010000,0.020000,0.000000,0.00,0.000000,0.020000,0.000000,0.010000,0.01000,0.010000,0.000000,0.010000,0.000000,0.000000,0.040000,0.010000,0.060000,0.000000,0.000000,0.000000,0.02,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.020000,0.000000,0.00,0.000000,0.000000,0.010000,0.040000,0.000000,0.00,0.000000,0.000000,0.010000,0.02,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.010000,0.020000,0.000000,0.040000,0.00,0.00,0.0000,0.01,0.040000,0.020000,0.000000,0.000000,0.020000,0.00,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.020000,0.000000,0.000000,0.01000,0.010000,0.00,0.000000,0.0000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.020000,0.000000,0.01000,0.000000,0.010000,0.000000,0.01,0.00,0.030000,0.01,0.010000,0.000000,0.010000,0.010000,0.010000,0.030000,0.000000,0.000000,0.00,0.0000,0.010000,0.000000,0.000000,0.01,0.020000,0.010000,0.000000,0.020000,0.000000,0.00,0.000000,0.00000,0.010000,0.000000,0.020000,0.000000,0.00,0.01,0.030000,0.01
1,CEDRITOS,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.00,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.045455,0.000000,0.000000,0.045455,0.045455,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.045455,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000

In [29]:
bogota_grouped.shape

(19, 153)

Create a function to display venues in descending order

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighbourhood.

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bogota_grouped['Neighborhood']

for ind in np.arange(bogota_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bogota_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ANTIGUO COUNTRY,Coffee Shop,Clothing Store,Italian Restaurant,Bar,Ice Cream Shop,French Restaurant,Wings Joint,Restaurant,Shopping Mall,Hotel
1,CEDRITOS,Pizza Place,Cocktail Bar,Nightclub,Sandwich Place,Restaurant,Racetrack,Pet Store,Coffee Shop,Dance Studio,Bike Rental / Bike Share
2,CHAPINERO ALTO,Restaurant,Bar,Gay Bar,Nightclub,Arepa Restaurant,Latin American Restaurant,Coffee Shop,Café,Breakfast Spot,Fast Food Restaurant
3,CHICO,Restaurant,Hotel,Breakfast Spot,Gym / Fitness Center,Juice Bar,Beer Garden,Salad Place,Latin American Restaurant,Italian Restaurant,French Restaurant
4,COLINA CAMPESTRE,Fast Food Restaurant,Plaza,Movie Theater,Gym / Fitness Center,Gym,Park,Sandwich Place,Women's Store,Donut Shop,Event Space
5,EMAUS,Coffee Shop,Hotel,Italian Restaurant,Restaurant,Park,Comfort Food Restaurant,Burger Joint,Burrito Place,Pub,Plaza
6,FONTIBON,Food & Drink Shop,Restaurant,Hotel,Shopping Mall,Gym,Fried Chicken Joint,Mexican Restaurant,Notary,Park,Clothing Store
7,GALERIAS,Bar,Burger Joint,Bakery,Restaurant,Fast Food Restaurant,Mexican Restaurant,Pizza Place,Nightclub,BBQ Joint,Pub
8,LA MACARENA,Restaurant,Sandwich Place,Peruvian Restaurant,Café,Ice Cream Shop,Coffee Shop,Spanish Restaurant,Tapas Restaurant,Bar,Italian Restaurant
9,LAS AMERICAS,Fast Food Restaurant,Multiplex,Bar,Sandwich Place,Restaurant,Burger Joint,Donut Shop,Coffee Shop,Breakfast Spot,Brewery


### Clustering Neighbourhoods in Bogota

### Step 1 - Run K Means to cluster neighbourhoods

In [32]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

bogota_grouped_clustering = bogota_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bogota_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 0, 4, 1, 4, 2, 0, 0, 0], dtype=int32)

### Step 2 - Create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bogota_merged = bogota_coordinates

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bogota_merged = bogota_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

bogota_merged

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,CHAPINERO,CHICO,4.681269,-74.046840,4,Restaurant,Hotel,Breakfast Spot,Gym / Fitness Center,Juice Bar,Beer Garden,Salad Place,Latin American Restaurant,Italian Restaurant,French Restaurant
1,CHAPINERO,CHAPINERO ALTO,4.645833,-74.063333,0,Restaurant,Bar,Gay Bar,Nightclub,Arepa Restaurant,Latin American Restaurant,Coffee Shop,Café,Breakfast Spot,Fast Food Restaurant
2,CHAPINERO,ANTIGUO COUNTRY,4.667285,-74.054726,4,Coffee Shop,Clothing Store,Italian Restaurant,Bar,Ice Cream Shop,French Restaurant,Wings Joint,Restaurant,Shopping Mall,Hotel
3,CHAPINERO,MARLY,4.637717,-74.064042,0,Fast Food Restaurant,Bar,Restaurant,Nightclub,Sandwich Place,Burger Joint,Bakery,Latin American Restaurant,Vegetarian / Vegan Restaurant,Market
4,CHAPINERO,EMAUS,4.651532,-74.055264,4,Coffee Shop,Hotel,Italian Restaurant,Restaurant,Park,Comfort Food Restaurant,Burger Joint,Burrito Place,Pub,Plaza
5,ENGATIVA,NORMANDIA,4.669283,-74.104953,4,Burger Joint,BBQ Joint,Sandwich Place,Bed & Breakfast,Breakfast Spot,Café,Cupcake Shop,Luggage Store,Fast Food Restaurant,Fried Chicken Joint
6,FONTIBON,FONTIBON,4.678611,-74.141111,2,Food & Drink Shop,Restaurant,Hotel,Shopping Mall,Gym,Fried Chicken Joint,Mexican Restaurant,Notary,Park,Clothing Store
7,FONTIBON,MODELIA,4.667180,-74.118589,0,Bar,Mexican Restaurant,Pizza Place,Restaurant,BBQ Joint,Ice Cream Shop,Cocktail Bar,Sandwich Place,Seafood Restaurant,Candy Store
8,KENNEDY,LAS AMERICAS,4.616046,-74.136947,0,Fast Food Restaurant,Multiplex,Bar,Sandwich Place,Restaurant,Burger Joint,Donut Shop,Coffee Shop,Breakfast Spot,Brewery
9,SANTA FE,SAN DIEGO,4.614470,-74.070100,0,Restaurant,Peruvian Restaurant,Café,Sandwich Place,Mexican Restaurant,Spanish Restaurant,Latin American Restaurant,Burger Joint,Tapas Restaurant,Coffee Shop


### Step 3 - Visualize the resulting clusters

In [34]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bogota_merged['Latitude'], bogota_merged['Longitude'], 
                                  bogota_merged['Neighbourhood'], bogota_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Step 4 - Examine each cluster and determine the discriminating venue categories that distinguish each cluster

#### Cluster 1

In [35]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 0, bogota_merged.columns[[1] + list(range(5, bogota_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,CHAPINERO ALTO,Restaurant,Bar,Gay Bar,Nightclub,Arepa Restaurant,Latin American Restaurant,Coffee Shop,Café,Breakfast Spot,Fast Food Restaurant
3,MARLY,Fast Food Restaurant,Bar,Restaurant,Nightclub,Sandwich Place,Burger Joint,Bakery,Latin American Restaurant,Vegetarian / Vegan Restaurant,Market
7,MODELIA,Bar,Mexican Restaurant,Pizza Place,Restaurant,BBQ Joint,Ice Cream Shop,Cocktail Bar,Sandwich Place,Seafood Restaurant,Candy Store
8,LAS AMERICAS,Fast Food Restaurant,Multiplex,Bar,Sandwich Place,Restaurant,Burger Joint,Donut Shop,Coffee Shop,Breakfast Spot,Brewery
9,SAN DIEGO,Restaurant,Peruvian Restaurant,Café,Sandwich Place,Mexican Restaurant,Spanish Restaurant,Latin American Restaurant,Burger Joint,Tapas Restaurant,Coffee Shop
10,LA MACARENA,Restaurant,Sandwich Place,Peruvian Restaurant,Café,Ice Cream Shop,Coffee Shop,Spanish Restaurant,Tapas Restaurant,Bar,Italian Restaurant
13,PALERMO,Café,Performing Arts Venue,New American Restaurant,Soup Place,Bookstore,Restaurant,Caribbean Restaurant,Brewery,Cocktail Bar,Mexican Restaurant
14,GALERIAS,Bar,Burger Joint,Bakery,Restaurant,Fast Food Restaurant,Mexican Restaurant,Pizza Place,Nightclub,BBQ Joint,Pub


In [36]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 1, bogota_merged.columns[[1] + list(range(5, bogota_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,COLINA CAMPESTRE,Fast Food Restaurant,Plaza,Movie Theater,Gym / Fitness Center,Gym,Park,Sandwich Place,Women's Store,Donut Shop,Event Space


In [37]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 2, bogota_merged.columns[[1] + list(range(5, bogota_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,FONTIBON,Food & Drink Shop,Restaurant,Hotel,Shopping Mall,Gym,Fried Chicken Joint,Mexican Restaurant,Notary,Park,Clothing Store


In [38]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 3, bogota_merged.columns[[1] + list(range(5, bogota_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,TEUSAQUILLO,Bakery,Department Store,Food Truck,Restaurant,Bowling Alley,Nightclub,Soccer Field,Theme Park,Coffee Shop,Dessert Shop


In [39]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 4, bogota_merged.columns[[1] + list(range(5, bogota_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,CHICO,Restaurant,Hotel,Breakfast Spot,Gym / Fitness Center,Juice Bar,Beer Garden,Salad Place,Latin American Restaurant,Italian Restaurant,French Restaurant
2,ANTIGUO COUNTRY,Coffee Shop,Clothing Store,Italian Restaurant,Bar,Ice Cream Shop,French Restaurant,Wings Joint,Restaurant,Shopping Mall,Hotel
4,EMAUS,Coffee Shop,Hotel,Italian Restaurant,Restaurant,Park,Comfort Food Restaurant,Burger Joint,Burrito Place,Pub,Plaza
5,NORMANDIA,Burger Joint,BBQ Joint,Sandwich Place,Bed & Breakfast,Breakfast Spot,Café,Cupcake Shop,Luggage Store,Fast Food Restaurant,Fried Chicken Joint
15,USAQUEN,Italian Restaurant,Burger Joint,Pub,Restaurant,Hotel,Steakhouse,Pizza Place,French Restaurant,Café,Dessert Shop
16,PEPE SIERRA,Bakery,Restaurant,Burger Joint,Italian Restaurant,Peruvian Restaurant,Coffee Shop,Gym / Fitness Center,Café,Park,American Restaurant
17,CEDRITOS,Pizza Place,Cocktail Bar,Nightclub,Sandwich Place,Restaurant,Racetrack,Pet Store,Coffee Shop,Dance Studio,Bike Rental / Bike Share
18,RINCON DEL CHICO,Hotel,Burger Joint,Italian Restaurant,Coffee Shop,Restaurant,Pub,Asian Restaurant,Japanese Restaurant,Gym / Fitness Center,Sandwich Place
